# RF SIE 1903 Prediction

This script was used for generating the SIE regression found in the _1903Prediction.nc_ file.

This regression was saved as an output as we decided that for now (31/08/2022) it would be useful to just have a base product of the current best prediction for all sea ice going back to the start of the 20th century.


##### From some prior investigation:
Using a monthly mode of prediction seems to work best in most models, so the most important thing was picking which years to use in the training period, as picking a longer period means more training data but fewer available proxies, e.g. Law Dome MSA available up until 1995, AP_stacked_MSA to 2002, WHG_dex to 2006... Training always starts in 1979 as those are the first satellite records.

2006 as the end year of the training period is the sweet spot for the current version of the model: Loss of Law Dome and AP  MSA proxies appears to be offset by the gains provided by having a longer training period. Progressing past 2006 we see error in the model increase again as further loss of proxies is not outweighed by gain of additional training data.



---
### Creating the model

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from calendar import month_abbr 

Grab in the list of month contractions and pop the empty string so it can be used in the graphs for axis tick labels.

In [ ]:
months = list(month_abbr)
months.pop(0)

Set the number of estimators (= trees in each forest), as well as seeding a random state for reproducibility.

In [ ]:
n_est = 200 #number of estimators
rs = 0 #random state
np.random.seed(rs)

Next we specify the years which will be used for:
- the start of the prediction period (predstartyear)
- the start of the training/testing data (startyear)
- the end of the training/testing data (endyear)
- the number of years we are predicting for (predyears)

In [ ]:
predstartyear = 1903 #start of predicted period
startyear = 1979 #start of training
endyear = 2006 #end of training
predyears = startyear-predstartyear

Then we specify filepaths and open the data for our prediction and training sections.

In [ ]:
yvar = 'SIE'
folder = '~/Desktop/IMAS/ncfiles/'  #filepath
pfile = 'Proxy_combined_data_v4.nc'  #proxy filename
xfile = 'CombinedProxies.nc'  #proxy filename
df = xr.open_dataset(folder+pfile).sel(year= slice(startyear, endyear))
X = xr.open_dataset(folder+xfile).sel(year= slice(predstartyear, endyear))

Then we drop predictors containing NaN values in this range.

__Note:__ We must do this as RFs cannot handle NaN values, there are many alternative options here such as:
- using the mean to impute values
- using a RF based on other proxies in that year (or only complete years) to impute
- using a known distribution to impute

In [ ]:
X = X.to_dataframe().dropna(axis=1).to_xarray()

This script is adapted from one with many modes of prediction, so get the output mode names in an array and set the mode to 1 as we are using _Monthly_ predictions for higher accuracy.

In [ ]:
outputname = ['Individual','Month','Longitudinal','Year']
mode = 1

Then log the mean SIE for use in calculating error%.

__Note:__ you could also calculate the mean SIE in each month, or at each longitude, for different ways of considering the error. I have used this as it was the simplest to implement in a short amount of time.

In [ ]:
mean_sie = df[yvar].mean('month').mean('lon').mean('year')

Next generate a train test split.

I am doing this here by randomly selecting arrays of years instead of using sklearn traintest split as not all the splits are done simultaneously and therefore can't all be done in one call of the function to maintain the same splits. So, I need to generate year lists here to ensure the years in train/test data are consistent  across X and y's. Note that the split is seeded by the numpy random seed above to be reproducible.

In [ ]:
years = np.arange(startyear, endyear+1)
testyears = np.sort(np.random.choice(years, size = int(len(years)*0.33), replace = False))
trainyears = [x for x in years if x not in testyears]

Select the correct section of predictor data for train/test/regression and then standard scale them.

In [ ]:
X_train = X.sel(year = trainyears).to_dataframe().values
X_test = X.sel(year = testyears).to_dataframe().values

#X_reg is the predictors in the regression time period
X_reg = X.sel(year = slice(predstartyear,startyear-1)).to_dataframe().values

#Standardise values of proxies
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_reg = sc.transform(X_reg)

Group the SIE into bins of 10 degree longitude sections for the model.

In [ ]:
y = df[yvar].groupby_bins('lon', np.arange(0,361,10)).mean()

Initialise an array to store the Root Mean Squared Error as Percentage of Mean (rmsepom) for the model on the test set.

In [ ]:
rmsepom = np.zeros([12,36])

Initialise an array to store the results of the regression for each year.

In [ ]:
regression = np.zeros([predyears,12,36])

Loop through the months, building each a model to predicit for all longitudes in that month. Log the model's error as well as the prediction for the predyears.

In [ ]:
for m in range(12): #loop over the months
    m += 1 #month index in xarray starts at 1
    monthdata = y.sel(month=m)
    y_train = monthdata.sel(year = trainyears).to_numpy()
    y_test = monthdata.sel(year = testyears).to_numpy()
    
    #Training model
    regressor = RandomForestRegressor(n_estimators = n_est, random_state = rs)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    
    y_reg = regressor.predict(X_reg) #yr=0 for 1903
    
    #Convert nested arrays from longitudes to year time series
    y_test = y_test.transpose()
    y_pred = y_pred.transpose() 
    
    for long in range(36):
        rmse = np.sqrt(metrics.mean_squared_error(y_test[long], y_pred[long]))
        rmsepom[m-1,long] = 100*rmse/mean_sie
    
    for yr in range(predyears):
        regression[yr,m-1] = y_reg[yr]

---
### Analysing the model

First, define a function to plot a heatmap of the error from the model stored in rmsepom. This shows the RMSEPOM at each month/longitude section for the model with a colour scale.

In [ ]:
def draw(rmsepom):
    ax = plt.subplot()
    
    rmsepomxr = xr.DataArray(rmsepom)
    rmsepomxr.plot.pcolormesh(levels = clev)
    
    ax.set_title('Error in '+yvar+' predictions using RFs')
    ax.set_ylabel('Month')
    #plt.yticks(ticks=np.arange(12))
    ax.set_xlabel('Longitude')
    ax.set_xticks(ticks=np.arange(3,36,3), labels=['','60E','','120E','','180','','120W','','60W',''])
    ax.set_yticks(ticks=np.arange(12), labels=months, rotation=0)
    ax.annotate("Mode = {}".format(outputname[mode]), xy = (0,0), xytext = (-6,-2.1))
    
    plt.show()

Next, define a function to plot the error distribution. This produces a histogram of the error values into 2% bins. It is also annotated with green, orange, and red lines to show where the 50th, 75th, and 95th percentiles of error respecitvely fall.

In [ ]:
def drawerror(rmsepom):
    ax = plt.subplot()
    
    ordered = rmsepom.copy()
    ordered.ravel().sort()
    
    x1 = np.percentile(ordered.ravel(), 50)
    plt.plot([x1, x1], [0, 100], color='g', linestyle='dashed', linewidth=2)
    ax.annotate("50%: {}".format(round(x1,1)), xy = (35,100), color = 'g')
    
    x1 = np.percentile(ordered.ravel(), 75)
    ax.annotate("75%: {}".format(round(x1,1)), xy = (35,92.5), color = 'tab:orange')
    plt.plot([x1, x1], [0, 100], color='tab:orange', linestyle='dashed', linewidth=2)
    
    x1 = np.percentile(ordered.ravel(), 95)
    plt.plot([x1, x1], [0, 100], color='r', linestyle='dashed', linewidth=2)
    ax.annotate("95%: {}".format(round(x1,1)), xy = (35,85), color='r')
    
    plt.hist(ordered.ravel(), bins = np.arange(0,41,2))
    
    ax.set_title('Error distribution for mode: {}'.format(outputname[mode]))
    ax.set_ylabel('Number of Values')
    ax.set_xlabel('Error%')
    ax.set_yticks(ticks=np.arange(0,120,10))
    ax.set_xticks(ticks=np.arange(0,41,2))
    ax.annotate("Estimators = {}".format(n_est), xy = (0,0), xytext = (-6,-20))
    
    plt.show()

Generate some generic metrics of the error.

In [ ]:
print("=========================================================")
print('Error in '+yvar+' predictions using RFs with Output Mode = {}'.format(outputname[mode]))
print('RMSE % minimum: ', rmsepom.min())
print('RMSE % maximum: ', rmsepom.max())
print('RMSE % mean: ', rmsepom.mean())
print("=========================================================")

Draw the heatmaps by setting up contour levels and calling the draw and drawerror functions.

In [ ]:
#Contour levels for 'zoomed out'
clev = np.arange(0,30,2)  #contour levels
draw(rmsepom)

#Contour levels for 5%
clev = np.arange(0,5,.25)  #contour levels
draw(rmsepom)

drawerror(rmsepom)

Next, load the data into a _prediction_ variable as an xarray DataArray and format it to have the correct coordinates.

In [ ]:
prediction = xr.DataArray(regression,attrs=dict(description=("Prediction of SIE 1903 to 1978")))
prediction = prediction.rename({'dim_0': 'year', 'dim_1':'month','dim_2':'lon'})
prediction = prediction.assign_coords(year = np.arange(1903,1979))
prediction = prediction.assign_coords(month = np.arange(1,13))
prediction = prediction.assign_coords(lon = np.arange(0,360,10))
prediction = prediction.to_dataset(name = 'SIE')

Print out the list of proxies used by this model.

In [ ]:
print('Proxies used are:')
print(list(X.keys()))

Finally, export the data to a netCDF file.

In [ ]:
filedirectory = r"C:\Users\Alfie\Desktop\IMAS\ncfiles\1903Prediction.nc"
prediction.to_netcdf(filedirectory)